## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os 
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from IPython.core.debugger import Tracer

%matplotlib inline

## Helper Functions

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


def region_of_interest(img, vertices):
   
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_cout
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines,color=[[255, 5, 5], [5, 255, 5]], thickness = 20):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    

    
    positive_line = []
    negative_line = []

    #Tracer()()
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope_m = (y2 - y1) / (x2 - x1) 
            
     #discard outliers
        if (0.5 < slope_m < 1.0) or (-1.0 < slope_m < -0.5): 
            
     # seperate x, y coordinates based on thier slope    
            if slope_m > 0:
        
     # calculate center x, y coordinates for each line segement 
    
                center_x = ((x2+x1)/2)    
                center_y = ((y2+y1)/2)
                intercept_b = y1 - slope_m * x1
                length = math.sqrt((x1-x2)**2 + (y1-y2)**2)
                positive_line.append((slope_m,center_x,center_y,intercept_b,length))
                
                
            elif slope_m < 0:
                center_x = ((x2+x1)/2)    
                center_y = ((y2+y1)/2)
                intercept_b = y1 - slope_m * x1
                length = math.sqrt((x1-x2)**2 + (y1-y2)**2)
                negative_line.append((slope_m,center_x,center_y,intercept_b,length)) 

    
    # calculate the average slope and center for all line segements
    # use length of line segements as weights. 
    
    # right line 
    positive_line = np.array(positive_line)
    right_line_slope = np.average(positive_line[:, 0]) 
    right_line_x1 = int(np.average(positive_line[:, 1],weights=positive_line[:, 4]))
    right_line_y1 = int(np.average(positive_line[:, 2],weights=positive_line[:, 4]))
    right_line_intercept = int(np.average(positive_line[:, 3],weights=positive_line[:, 4]))
    
    # left line 
    negative_line = np.array(negative_line)
    left_line_slope = np.average(negative_line[:, 0]) 
    left_line_x1 = int(np.average(negative_line[:, 1],weights= negative_line[:, 4])) 
    left_line_y1 = int(np.average(negative_line[:, 2],weights= negative_line[:, 4])) 
    left_line_intercept = int(np.average(negative_line[:, 3],weights= negative_line[:, 4])) 
    

    image_shape = img.shape
    
    # determine x snd y coordinates based on (y2-y1) = m (x2-x1) equation 
    
    left_x2 = int((image_shape[0]-left_line_y1)/left_line_slope)+left_line_x1
    right_x2 = int((image_shape[0]-right_line_y1)/right_line_slope)+right_line_x1
    
    
    # use a fixed Y point and calculate new x coordinate 
    left_line_y1=330
    new_left_x1 = int(left_x2 - ((image_shape[0]-left_line_y1)/left_line_slope))

    right_line_y1=330
    new_right_x1 = int(right_x2 - ((image_shape[0]-right_line_y1)/right_line_slope))
    
    # draw lines on the image 
    
    cv2.line(img,  (new_left_x1, left_line_y1),(left_x2, image_shape[0]),color[0], thickness)
    cv2.line(img,  (new_right_x1, right_line_y1),(right_x2, image_shape[0]), color[0], thickness)
    
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
   
    
    #print(lines)
   
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    draw_lines(line_img, lines)
    
    return line_img



In [3]:
def weighted_img(img, initial_img, α=1.0, β=0.5, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [4]:
# this is the pipe line that processes all the images sequentially 

def process_image(image):
            
        # convert the image to grayscale
            gray = grayscale(image)
        
            # Canny edge detection
            canny_image = canny(gray, 50, 150)
            
            
             # Gaussian Blur
            gauss_image = gaussian_blur(canny_image, 5)
            
            
            imshape = image.shape
            vertices = np.array([[(0,imshape[0]),(430, 330), (520, 330), (imshape[1],imshape[0])]], dtype=np.int32)
            
            # masked image edges for region of interest
            masked_image = region_of_interest(gauss_image, vertices)

            rho = 1 # distance resolution in pixels of the Hough grid
            theta = np.pi/180 # angular resolution in radians of the Hough grid
            threshold = 15     # minimum number of votes (intersections in Hough grid cell)
            min_line_len = 40 #minimum number of pixels making up a line
            max_line_gap = 20    # maximum gap in pixels between connectable line segments
            line_image = np.copy(image)*0 # creating a blank to draw lines on
            
            # plot hough lines

            hough_image = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
            #plt.imshow(hough_image)
            # overlay line image with original mage
            final_image = weighted_img(hough_image, image)
            
            
            #lines_edges = cv2.addWeighted(image, 0.8, hough_image, 1, 0) 

            return final_image
        
        
        
#def read_image_files():
   
def mark_lane_lines():
    path = "test_images/"

    # read files from a directory
    
    for filename in os.listdir("test_images/"):

        file_path=path+filename
        image = mpimg.imread(file_path)
        transformed_image = process_image(image) 
        out_file_name = "test_images/" + "output_"+filename
        cv2.imwrite(out_file_name, cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
    
#starting point for reading images and calling pipeline functions   
mark_lane_lines()  

In [5]:
white_output = 'Output_white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video Output_white.mp4
[MoviePy] Writing video Output_white.mp4


100%|█████████▉| 221/222 [00:03<00:00, 64.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_white.mp4 

CPU times: user 3.39 s, sys: 721 ms, total: 4.11 s
Wall time: 3.82 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [7]:
yellow_output = 'Output_yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video Output_yellow.mp4
[MoviePy] Writing video Output_yellow.mp4


100%|█████████▉| 681/682 [00:11<00:00, 58.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_yellow.mp4 

CPU times: user 11.3 s, sys: 2.25 s, total: 13.6 s
Wall time: 12.1 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

### Pipeline Description

####  Steps:
1. Read the images from the test_image folder
2. Gray Scale the image and apply Gaussian smoothing
3. Specify vertices and mark the region of interest using fillPoly() function
5. Define Hough transform parameters and run Hough transform on masked edge-detected image
6. Extrapolate line segements and draw complete lines. 
7. Combine line image with original image.


1. Spent majority of the time in creating draw line function. This function extrapolates the line segements to draw complete left and right lines on the lanes. 

2. The approach to draw the lines was based on identifying and bucketing positive and negative slopes for each line segement.

3. x and y coordinates are determined based on calcuating the center of the line and then using the equation (y2-y1)+m(x2-x1) to calculate the missing coordinate. 

4. if the line segements are too short(calcuated based on the slope < 0.5 ) they are ignored and only line segements that fall between 0.5 to 1 are choosen to eliminate noise. 






### Shortcomings

1. The vertices and image dimensions are hard coded in a few places.
2. In the video, the lines do not appear to be smooth all the time. This indicates that top and bottom coordinates between frames have noise that should be further corrected. 

### Possible Improvements

1. It may be possible to smoothen out the lines on videos by averaging the x & y coordinates from one frame to the next. 


